In [298]:
import pymysql.cursors
import pandas as pd
import numpy as np
import connect_to_db as cn
import json
from itertools import combinations
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import csv

In [299]:
def extract_bodies(author, table):
    # 추출해 낸 author를 이용해 comment body만 끌어옴.
    connection = cn.make_connection('connection.csv')
    cursor = connection.cursor(pymysql.cursors.DictCursor)
    sql = f"select body from {table} where author = '{author}';"

    cursor.execute(sql)
    result = cursor.fetchall()
    connection.close()

    # 추출된 body 목록을 list 형태로 바꾸어서 return.
    result_df = pd.DataFrame(result)
    result_list = list(np.array(result_df['body'].tolist()))
    
    return result_list

In [300]:
def check_body(document):
    # 특수문자, 이모지 등으로만 이루어진 comment/post를 거르기 위한 code.
    p = re.compile('[a-zA-Z]')
    for text in document[:]:
        if p.search(text) is None:
            document.remove(text)
        else:
            if len(text) == 1:
                document.remove(text)
    
    return document

In [301]:
def tf_idf_similarity(document):
    document = check_body(document)

    # comment/body list가 빈 list인지를 체크해서 빈 list이면 -1 return.
    if not document:
        return -1
    
    if len(document) == 1:
        return -2

    tfidf_vectorizer = TfidfVectorizer(min_df=1)
    tfidf_matrix = tfidf_vectorizer.fit_transform(document)

    document_distances = (tfidf_matrix * tfidf_matrix.T)
    
    result_array = document_distances.toarray()
    
    similarity = 0
    count = 0
    
    for i in range(len(result_array)):
        for j in range(len(result_array[i])):
            if i < j:
                similarity += result_array[i][j]
                count += 1
    
    return (similarity / count)

In [302]:
def write_similarity_csv(column, result_list):
    fields = ['author', column]
    filename = column + '.csv'
    with open(filename, 'w', newline='') as f:
        write = csv.writer(f)
        write.writerow(fields)
        write.writerows(result_list)

In [305]:
connection = cn.make_connection('connection.csv')

cursor = connection.cursor(pymysql.cursors.DictCursor)

sql = 'select author from mentor_new where comment_cnt > 1 and is_valid = 1 and comment_similarity is null limit 1000;'
cursor.execute(sql)

# result가 dict 형태로 return.
result = cursor.fetchall()
connection.close()

author_similarity_list = []

connection = cn.make_connection('connection.csv')
cursor = connection.cursor(pymysql.cursors.DictCursor)

result_df = pd.DataFrame(result)
# author column의 value들로만 이루어진 np list 생성.
np_result = np.array(result_df['author'].astype(str).values.tolist())

result_for_csv = []

for author in np_result:
    bodies = extract_bodies(author, 'comments')
    similarity = tf_idf_similarity(bodies)
    result_for_csv.append([author, similarity])

write_similarity_csv('comment_similarity', result_for_csv)

connection.close()